---
# Data Science and Artificial Intelliegence Practicum
## 5-modul. Machine Learning
---

## 5.16 - Hierarchical Clustering

### Imports and loading data

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler

Cars dataset.

In [2]:
URL = "https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IBMDeveloperSkillsNetwork-ML0101EN-SkillsNetwork/labs/Module%204/data/cars_clus.csv"
df = pd.read_csv(URL)
df.head()

,manufact,model,sales,resale,type,price,engine_s,horsepow,wheelbas,width,length,curb_wgt,fuel_cap,mpg,lnsales,partition
0,Acura,Integra,16.919,16.360,0.000,21.500,1.800,140.000,101.200,67.300,172.400,2.639,13.200,28.000,2.828,0.0
1,Acura,TL,39.384,19.875,0.000,28.400,3.200,225.000,108.100,70.300,192.900,3.517,17.200,25.000,3.673,0.0
2,Acura,CL,14.114,18.225,0.000,$null$,3.200,225.000,106.900,70.600,192.000,3.470,17.200,26.000,2.647,0.0
3,Acura,RL,8.588,29.725,0.000,42.000,3.500,210.000,114.600,71.400,196.600,3.850,18.000,22.000,2.150,0.0
4,Audi,A4,20.397,22.255,0.000,23.990,1.800,150.000,102.600,68.200,178.000,2.998,16.400,27.000,3.015,0.0


### Data Exploration

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159 entries, 0 to 158
Data columns (total 16 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   manufact   157 non-null    object 
 1   model      159 non-null    object 
 2   sales      159 non-null    object 
 3   resale     159 non-null    object 
 4   type       159 non-null    object 
 5   price      159 non-null    object 
 6   engine_s   159 non-null    object 
 7   horsepow   159 non-null    object 
 8   wheelbas   159 non-null    object 
 9   width      159 non-null    object 
 10  length     159 non-null    object 
 11  curb_wgt   159 non-null    object 
 12  fuel_cap   159 non-null    object 
 13  mpg        159 non-null    object 
 14  lnsales    159 non-null    object 
 15  partition  159 non-null    float64
dtypes: float64(1), object(15)
memory usage: 20.0+ KB


- Non-numeric types.
- Missing values.

### Data Preprocessing

`errors='coerce'` parameter in `pd.to_numeric` function means that if a value in the columns cannot be converted to a numeric type, then it will be replaced with `NaN`instead of raising an error.

In [4]:
df.sample()

,manufact,model,sales,resale,type,price,engine_s,horsepow,wheelbas,width,length,curb_wgt,fuel_cap,mpg,lnsales,partition
154,Volvo,V70,17.531,$null$,0.000,28.800,2.400,168.000,104.900,69.300,186.200,3.259,17.900,25.000,2.864,0.0


In [5]:
df[['sales', 'resale', 'type', 'price', 'engine_s', 'horsepow',
    'wheelbas', 'width', 'length', 'curb_wgt', 'fuel_cap','mpg',
    'lnsales']] = df[['sales', 'resale', 'type', 'price', 'engine_s',
    'horsepow', 'wheelbas', 'width', 'length', 'curb_wgt', 'fuel_cap', 'mpg',
    'lnsales']].apply(pd.to_numeric, errors='coerce')
df.head()

,manufact,model,sales,resale,type,price,engine_s,horsepow,wheelbas,width,length,curb_wgt,fuel_cap,mpg,lnsales,partition
0,Acura,Integra,16.919,16.360,0.0,21.50,1.8,140.0,101.2,67.3,172.4,2.639,13.2,28.0,2.828,0.0
1,Acura,TL,39.384,19.875,0.0,28.40,3.2,225.0,108.1,70.3,192.9,3.517,17.2,25.0,3.673,0.0
2,Acura,CL,14.114,18.225,0.0,NaN,3.2,225.0,106.9,70.6,192.0,3.470,17.2,26.0,2.647,0.0
3,Acura,RL,8.588,29.725,0.0,42.00,3.5,210.0,114.6,71.4,196.6,3.850,18.0,22.0,2.150,0.0
4,Audi,A4,20.397,22.255,0.0,23.99,1.8,150.0,102.6,68.2,178.0,2.998,16.4,27.0,3.015,0.0


In [6]:
df.isnull().sum() / len(df) * 100

manufact      1.257862
model         0.000000
sales         1.257862
resale       23.899371
type          1.257862
price         1.257862
engine_s      0.628931
horsepow      0.628931
wheelbas      0.628931
width         0.628931
length        0.628931
curb_wgt      1.257862
fuel_cap      0.628931
mpg           1.886792
lnsales       1.257862
partition     0.000000
dtype: float64

The column `resale` has signifcianly (24%) `NaN` values.

In [7]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
sales,157.0,52.998076,68.029422,0.110,14.114,29.450,67.956,540.561
resale,121.0,18.072975,11.453384,5.160,11.260,14.180,19.875,67.550
type,157.0,0.261146,0.440665,0.000,0.000,0.000,1.000,1.000
price,157.0,27.396605,14.277567,9.235,18.145,22.799,31.965,85.500
engine_s,158.0,3.053797,1.045989,1.000,2.300,3.000,3.500,8.000
horsepow,158.0,185.132911,57.031459,55.000,148.500,175.000,215.000,450.000
wheelbas,158.0,107.494304,7.595307,92.600,103.025,107.000,112.200,138.700
width,158.0,71.155063,3.454759,62.600,68.400,70.550,73.475,79.900
length,158.0,187.272152,13.382310,149.400,177.525,187.900,196.050,224.500
curb_wgt,157.0,3.382382,0.633938,1.895,2.967,3.342,3.821,5.572


Roughly 75% of the column `resale` is smaller than 20. So we can fill `NaN` values with mean(~18).

**Selecting columns for clustering.**

When selecting features for hierarchical clustering, it's important to consider the domain knowledge and the goals of the analysis. Here are some general guidelines:

- Exclude irrelevant features: If a feature has no impact on the similarity/distance between data points, then it's irrelevant for clustering. For example, the "partition" column in your dataset seems to have no impact on the similarity of cars and can be excluded.

- Exclude redundant features: If two or more features provide the same information, then it's redundant to include all of them. For example, the "engine_s" and "horsepow" columns in your dataset seem to be highly correlated, so you can choose to keep only one of them.

- Normalize numerical features: If the numerical features have different scales, it's important to normalize them to avoid features with larger scales dominating the distance calculation. For example, in your dataset, the "price" feature has a much larger scale than the "mpg" feature, so you may want to normalize them before clustering.

- Handle missing values: If there are missing values in the dataset, you can choose to either impute them or exclude the corresponding data points. SimpleImputer from scikit-learn can be used to impute missing values.

- Perform feature selection: If the dataset has many features and some of them are noisy or irrelevant, it may be helpful to perform feature selection before clustering. Principal Component Analysis (PCA) or SelectKBest from scikit-learn can be used for feature selection.

It's important to note that the choice of features and their preprocessing can have a significant impact on the clustering results, so it's recommended to try different combinations and compare the results.

In [8]:
# df.corr(numeric_only=True).style.background_gradient("Blues")

In [9]:
# data = df[['']]

In [10]:
# imputer = SimpleImputer(strategy='mean')
# data_imputed = imputer.fit_transform()
# data_imputed[:5]

**Standardization**

In [11]:
# scaler = MinMaxScaler()
# data_mtx = scaler.fit_transform(data_imputed)
# data_mtx[:5]

In [12]:
# data = pd.DataFrame(data_mtx, columns=data.columns)
# data.head()

### Clustering

**Distance matrix** -

Building distance matrix.